In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install import_ipynb 
!conda install -c conda-forge import_ipynb

/Users/gabesmithline/miniconda3/lib/python3.10/site-packages/conda/base/context.py:200: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(
Channels:
 - conda-forge
 - defaults
 - pytorch
Platform: osx-arm64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - import_ipynb

Current channels:

  - https://conda.anaconda.org/conda-forge
  - defaults
  - https://conda.anaconda.org/pytorch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at

In [3]:
import sys

sys.path.append('../')

from game_runner import NegotitaionGame
from eval.game_evaluator import GameEvaluator
import agents.simple_agent as simple_agent
import agents.llm_agent as llm_agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dataclasses import dataclass, field
from math import prod, sqrt
sys.path.append('../caif_negotiation/')

# Import the notebook
#import import_ipynb
#%run '../test_game_eval.ipynb'  
import torch
from utils.offer import Offer

from prompts.make_prompt import make_prompt
from prompts.make_prompt_bargain import make_prompt_bargain
from metrics.visualizations import (
    plot_discounted_values,
    plot_offer_evolution,
    plot_negotiation_gap,
    plot_fairness
)
import concurrent.futures


pathology_results = pd.DataFrame()  
import itertools
envy_results_history = {}
from eval.metrics import *
from utils.helpers import *
from utils.negotiation_game import *



Matplotlib created a temporary cache directory at /var/folders/fh/fwc37qhn04d8sxp65hwv1kxm0000gn/T/matplotlib-ka8438d2 because the default path (/Users/gabesmithline/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.


Outside offer consistency: True
Offer increasing: False
Envy-free (strict): True
Envy-free (excluding one item): True

Player 1:
Outside offer value: 50.0
Offer 1:
  Offer: tensor([1, 1, 2, 0])
  Player values: tensor([10, 20, 30, 40])
  Value calculation:
    Total value: 230 = tensor([10, 20, 30, 40]) · tensor([3, 2, 4, 1])
    Given value: 90 = tensor([10, 20, 30, 40]) · tensor([1, 1, 2, 0])
    Kept value: 140 = 230 - 90
  Value of kept items: 140
Offer 2:
  Offer: tensor([2, 1, 3, 0])
  Player values: tensor([10, 20, 30, 40])
  Value calculation:
    Total value: 230 = tensor([10, 20, 30, 40]) · tensor([3, 2, 4, 1])
    Given value: 130 = tensor([10, 20, 30, 40]) · tensor([2, 1, 3, 0])
    Kept value: 100 = 230 - 130
  Value of kept items: 100

Player 2:
Outside offer value: 40.0
Offer 1:
  Offer: tensor([2, 1, 2, 1])
  Player values: tensor([40, 30, 20, 10])
  Value calculation:
    Total value: 270 = tensor([40, 30, 20, 10]) · tensor([3, 2, 4, 1])
    Given value: 160 = tensor([

In [4]:
import time
import pandas as pd
import torch
import numpy as np
from math import sqrt, prod

# ------------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------------
# prompt_style = 'llama_3.3_70b_maximize_value_outside_offer_cot_1_test'
prompt_style = 'openai_o1_test_new_data_object_parallel'
llm_type = 'openai'
date = '1_28_2025'
max_rounds = 3
games = 10
circles = [5, 6]



## Run Games

In [5]:
'''
for circle in circles:
    print(f"Running game for circle {circle}")
    run_game(circle, games, max_rounds, date, prompt_style, llm_type)
'''

with concurrent.futures.ProcessPoolExecutor() as executor:
    future_to_circle = {
        executor.submit(run_game, circle, games, max_rounds, date, prompt_style, llm_type): circle
        for circle in circles
    }
    for future in concurrent.futures.as_completed(future_to_circle):
        circle_val = future_to_circle[future]
        try:
            future.result()
            print(f"[INFO] circle={circle_val} run finished successfully.")
        except Exception as exc:
            print(f"[ERROR] circle={circle_val} generated an exception: {exc}")

Outside offer consistency: True
Offer increasing: False
Envy-free (strict): True
Envy-free (excluding one item): True

Player 1:
Outside offer value: 50.0
Offer 1:
  Offer: tensor([1, 1, 2, 0])
  Player values: tensor([10, 20, 30, 40])
  Value calculation:
    Total value: 230 = tensor([10, 20, 30, 40]) · tensor([3, 2, 4, 1])
    Given value: 90 = tensor([10, 20, 30, 40]) · tensor([1, 1, 2, 0])
    Kept value: 140 = 230 - 90
  Value of kept items: 140
Offer 2:
  Offer: tensor([2, 1, 3, 0])
  Player values: tensor([10, 20, 30, 40])
  Value calculation:
    Total value: 230 = tensor([10, 20, 30, 40]) · tensor([3, 2, 4, 1])
    Given value: 130 = tensor([10, 20, 30, 40]) · tensor([2, 1, 3, 0])
    Kept value: 100 = 230 - 130
  Value of kept items: 100

Player 2:
Outside offer value: 40.0
Offer 1:
  Offer: tensor([2, 1, 2, 1])
  Player values: tensor([40, 30, 20, 10])
  Value calculation:
    Total value: 270 = tensor([40, 30, 20, 10]) · tensor([3, 2, 4, 1])
    Given value: 160 = tensor([

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

def analyze_bargaining_statistics(csv_path):
    """
    Reads the CSV of game results (including welfare and bargaining stats),
    computes aggregate measures by step, and visualizes them in separate plots.
    
    Args:
        csv_path (str): Path to the CSV file.
    """

    # 1. Read the CSV into a DataFrame
    df = pd.read_csv(csv_path)

    df = df.replace({None: np.nan, 'None': np.nan})

    df['round_num'] = pd.to_numeric(df['round_num'], errors='coerce')
    df['player'] = pd.to_numeric(df['player'], errors='coerce')

    df['step_id'] = df['round_num'] + 0.5 * (df['player'] - 1)
    df['step_id'] = pd.to_numeric(df['step_id'], errors='coerce')  # Ensure step_id is float

    # 5. Debugging: Print sample 'step_id' values
    print("Sample 'step_id' values:")
    print(df[['round_num', 'player', 'step_id']].head())

    # 6. Specify the numeric columns to analyze and visualize
    columns_of_interest = [
        "game_items",
        "offer", 
        "your_side_of_current_offer",
        "on_pareto_frontier",
        "value",
        "undiscounted_value",
        "nash_welfare",
        "proposal_proportion_player_1",
        "proposal_proportion_player_2",
        "concession_size",
        "security_level_player_1",
        "security_level_player_2",
        "rawlsian_welfare",
        "gini_coefficient",
        "utilitarian_welfare",
        "jain_fairness_index",
    ]
    
    # 7. Convert relevant columns to numeric, coerce errors -> NaN
    for col in columns_of_interest:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 8. Sort DataFrame by 'step_id' to ensure proper ordering
    df = df.sort_values('step_id')

    # 9. Compute summary statistics by step_id (mean, std, etc.)
    group_by_step = df.groupby("step_id")
    summary_stats = {}
    for col in columns_of_interest:
        if col in df.columns:
            summary_stats[col] = {
                "mean": group_by_step[col].mean(),
                "std":  group_by_step[col].std(),
                "min":  group_by_step[col].min(),
                "max":  group_by_step[col].max()
            }
    print(f"Counts of step_ids: {df['step_id'].value_counts()}")
    # 10. Print out some aggregated summaries
    print("=== AGGREGATE STATISTICS BY STEP ===")
    print("Step |  Column                           |  Mean    |  Std     |  Min     |  Max")
    print("----------------------------------------------------------------------------------")
    unique_steps = sorted(df["step_id"].dropna().unique())
    print(f"Unique step_ids: {unique_steps}")  # Debugging
    for step in unique_steps:
        for col in columns_of_interest:
            if col in df.columns:
                col_stats = summary_stats[col]
                mean_val = col_stats["mean"].get(step, np.nan)
                std_val  = col_stats["std"].get(step, np.nan)
                min_val  = col_stats["min"].get(step, np.nan)
                max_val  = col_stats["max"].get(step, np.nan)
                print(f"{step:<5} | {col:<32} | {mean_val:8.2f} | {std_val:8.2f} | {min_val:8.2f} | {max_val:8.2f}")

    # 11. Create separate line plots for each column vs. step_id
    #count on pareto frontier
    on_pareto_frontier_count = df[df['on_pareto_frontier'] == True].groupby('step_id').size()
    print(f"Counts of on_pareto_frontier: {on_pareto_frontier_count}")
    columns_exist = [col for col in columns_of_interest if col in df.columns]

    #pareto optimal game ending
    pareto_optimal_game_ending = df[df['on_pareto_frontier'] == True].groupby('').size()
    print(f"Counts of pareto_optimal_game_ending: {pareto_optimal_game_ending}")
    
    for col in columns_exist:
        if col in ["value", "undiscounted_value"]:
            # Create a new figure for each plot
            plt.figure(figsize=(10, 6))
            
            # Filter data for Player 1 and Player 2
            player1 = df[df['player'] == 1]
            player2 = df[df['player'] == 2]
            
            # Group by step_id and compute mean
            player1_grouped = player1.groupby('step_id')[col].mean().reindex(unique_steps, fill_value=np.nan)
            player2_grouped = player2.groupby('step_id')[col].mean().reindex(unique_steps, fill_value=np.nan)
            
            # Plot Player 1
            plt.plot(player1_grouped.index, player1_grouped.values, label='Player 1', color='blue', linestyle='-', marker='o')
            # Overlay square markers for Player 1 when it's Player 2's move (step_ids 1.5, 2.5, 3.5)
            player1_other_moves = player1_grouped.loc[player1_grouped.index % 1 != 0]
            plt.scatter(player1_other_moves.index, player1_other_moves.values, color='blue', marker='s', label='Player 1 Other\'s Move')
            
            # Plot Player 2
            plt.plot(player2_grouped.index, player2_grouped.values, label='Player 2', color='red', linestyle='-', marker='o')
            # Overlay square markers for Player 2 when it's Player 1's move (step_ids 1, 2, 3)
            player2_other_moves = player2_grouped.loc[player2_grouped.index % 1 == 0]
            plt.scatter(player2_other_moves.index, player2_other_moves.values, color='red', marker='s', label='Player 2 Other\'s Move')
            
            # Set title and labels
            plt.title(f"Mean {col.replace('_', ' ').title()} by Step")
            plt.xlabel("Step ID (e.g., 1 for Player 1 Round 1, 1.5 for Player 2 Round 1)")
            plt.ylabel(col.replace('_', ' ').title())
            
            # Set x-ticks and labels
            step_labels = [f"{int(step)}" if step.is_integer() else f"{step:.1f}" for step in unique_steps]
            plt.xticks(unique_steps, step_labels, rotation=45, ha='right')
            
            # Add grid
            plt.grid(True, linestyle='--', alpha=0.7)
            
            # Create custom legend to avoid duplicate labels
            custom_lines = [
                Line2D([0], [0], color='blue', lw=2, label='Player 1'),
                Line2D([0], [0], color='red', lw=2, label='Player 2'),
                Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=8, label='Own Move'),
                Line2D([0], [0], marker='s', color='w', markerfacecolor='blue', markersize=8, label='Other\'s Move (Player 1)'),
                Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8, label='Own Move'),
                Line2D([0], [0], marker='s', color='w', markerfacecolor='red', markersize=8, label='Other\'s Move (Player 2)'),
            ]
            plt.legend(handles=custom_lines, loc='best')
            
            # Adjust layout to prevent clipping of tick-labels
            plt.tight_layout()
            
            # Show the plot
            plt.show()
        else:
            # For other columns, plot as before
            # Create a new figure for each plot
            plt.figure(figsize=(10, 6))
            
            # Group by step_id and compute mean
            aggregated = group_by_step[col].mean().reindex(unique_steps, fill_value=np.nan)
    
            # Plot the data
            plt.plot(aggregated.index, aggregated.values, marker='o', linestyle='-', color='green', label='Mean Value')
            plt.title(f"Mean {col.replace('_', ' ').title()} by Step")
            plt.ylabel(col.replace('_', ' ').title())
            plt.xlabel("Step ID (e.g., 1 for Player 1 Round 1, 1.5 for Player 2 Round 1)")
            
            # Set x-ticks and labels
            step_labels = [f"{int(step)}" if step.is_integer() else f"{step:.1f}" for step in unique_steps]
            plt.xticks(unique_steps, step_labels, rotation=45, ha='right')
            
            # Add grid for better readability
            plt.grid(True, linestyle='--', alpha=0.7)
            
            # Add legend
            plt.legend()
            
            # Adjust layout to prevent clipping of tick-labels
            plt.tight_layout()
            
            # Show the plot
            plt.show()

#analyze_bargaining_statistics(f"../experiments/pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv")
analyze_bargaining_statistics(f"../experiments/pathology_results_1_23_2025_100_openai_o1_maximize_value_outside_offer_cot_1_test2_circle_0.csv")

Sample 'step_id' values:
   round_num  player  step_id
0          1       1      1.0
1          1       2      1.5
2          2       1      2.0
3          2       2      2.5
4          1       1      1.0
Counts of step_ids: step_id
1.0    100
1.5     95
2.0     61
2.5     26
3.0     18
Name: count, dtype: int64
=== AGGREGATE STATISTICS BY STEP ===
Step |  Column                           |  Mean    |  Std     |  Min     |  Max
----------------------------------------------------------------------------------
Unique step_ids: [1.0, 1.5, 2.0, 2.5, 3.0]
1.0   | offer                            |      nan |      nan |      nan |      nan
1.0   | your_side_of_current_offer       |      nan |      nan |      nan |      nan
1.0   | on_pareto_frontier               |     0.03 |     0.17 |     0.00 |     1.00
1.0   | value                            |   729.07 |   305.32 |   175.00 |  1734.00
1.0   | undiscounted_value               |   729.07 |   305.32 |   175.00 |  1734.00
1.0   | nash_welf

KeyError: ''

## Statistics of Play for Each Player Over Games

In [ ]:
# Read the CSV data
import pandas as pd
#/Users/gabesmithline/Desktop/caif_negotiation/experiments/2025_1_5/llama/basic_maximize_value_outside_offer_cot/pathology_results_1_5_2025_100_llama_3.1_405b_maximize_value_outside_offer_cot_.9.csv
df = pd.read_csv(f'../experiments/pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv')
#df = pd.read_csv("/Users/gabesmithline/Desktop/caif_negotiation/experiments/2025_1_4/llama/basic_maximize_value_outside_offer_cot/pathology_results_1_4_2025_100_llama_3.1_405b_maximize_value_outside_offer_cot_1.csv")
def analyze_pathology_comprehensive(df, pathology_name, is_game_ending_pathology=False):
    total_games = df['game_num'].nunique()

    if is_game_ending_pathology:
        # Identify moves where the pathology occurred
        #pathology_moves = df[df[pathology_name]]
        pathology_moves = df[df[pathology_name].fillna(False)]

        # Get the first occurrence per game (since game ends after pathology)
        first_pathology_moves = pathology_moves.groupby('game_num').first().reset_index()

        # Overall count of games with the pathology
        overall_count = len(first_pathology_moves)

        print(f"\n=== {pathology_name} ===")
        print(f"\nOverall: {overall_count}/{total_games} ({(overall_count/total_games*100):.1f}%)")

        # Count per player
        player_counts = first_pathology_moves['player'].value_counts()

        for player in [1, 2]:
            player_count = player_counts.get(player, 0)
            print(f"Player {player}: {player_count}/{total_games} ({(player_count/total_games*100):.1f}%)")

        # Count per round
        round_counts = first_pathology_moves['round_num'].value_counts()
        for round_num in sorted(df['round_num'].unique()):
            round_count = round_counts.get(round_num, 0)
            print(f"Round {round_num}: {round_count}/{total_games} ({(round_count/total_games*100):.1f}%)")

    else:
        # Identify the first occurrence of a game-ending pathology per game
        game_ending_moves = df[df[['accepting_an_offer_worse_than_outside_offer',
                                   'walking_away_from_an_offer_better_than_your_outside_offer']].any(axis=1)]
        #first_game_ending_round = game_ending_moves.groupby('game_num')['round_num'].min()
        first_game_ending_indices = game_ending_moves.groupby('game_num').first().reset_index()['round_num']
        # Function to get valid moves (before any game-ending pathology occurs)
        def get_valid_moves(group):
            game_num = group['game_num'].iloc[0]
            if game_num in first_game_ending_indices.index:
                #pathology_round = first_game_ending_round.loc[game_num]
                first_pathology_index = first_game_ending_indices.loc[game_num]

                #return group[group['round_num'] < pathology_round]  # Exclude the pathology round and after
                return group[group['round_num'] < first_pathology_index]  # Exclude the pathology round and after
            else:
                return group  

        # Apply the function to get all valid moves before game-ending pathologies
        valid_moves = df.groupby('game_num').apply(get_valid_moves).reset_index(drop=True)

        total_moves = len(valid_moves)
        count = valid_moves[pathology_name].sum()

        print(f"\n=== {pathology_name} ===")
        print(f"\nOverall: {int(count)}/{total_moves} ({(count/total_moves*100):.1f}%)")

        # By Player
        for player in [1, 2]:
            player_moves = valid_moves[valid_moves['player'] == player]
            player_count = player_moves[pathology_name].sum()
            moves = len(player_moves)
            if moves > 0:
                print(f"Player {player}: {int(player_count)}/{moves} ({(player_count/moves*100):.1f}%)")

        # By Round
        for round_num in sorted(valid_moves['round_num'].unique()):
            round_moves = valid_moves[valid_moves['round_num'] == round_num]
            round_count = round_moves[pathology_name].sum()
            moves_in_round = len(round_moves)
            if moves_in_round > 0:
                print(f"Round {round_num}: {int(round_count)}/{moves_in_round} ({(round_count/moves_in_round*100):.1f}%)")

# Define pathology types
game_ending_pathologies = [
    'accepting_an_offer_worse_than_outside_offer',
    'walking_away_from_an_offer_better_than_your_outside_offer'
]

per_move_pathologies = [
    'making_an_offer_worse_for_you_than_your_outside_offer',
    'offer_no_items_or_all_items',
    'making_offer_worse_than_previous'
]

# Analyze each pathology
for pathology in game_ending_pathologies:
    analyze_pathology_comprehensive(df, pathology, is_game_ending_pathology=True)

for pathology in per_move_pathologies:
    analyze_pathology_comprehensive(df, pathology, is_game_ending_pathology=False)

## Compute Bargaining Metrics

In [ ]:
df = pd.read_csv(f'../experiments/pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def analyze_bargaining_statistics(csv_path):
    """
    Reads the CSV of game results (including welfare and bargaining stats),
    computes aggregate measures by round, and visualizes them.
    
    Args:
        csv_path (str): Path to the CSV file (e.g. "pathology_results_1_18_2025_3_openai_o1_maximize_value_outside_offer_cot_1_test.csv")
    """

    # 1. Read the CSV into a DataFrame
    df = pd.read_csv(csv_path)

    # 2. Replace direct "None" strings or actual None with np.nan
    #    so that pandas can skip them appropriately in calculations.
    df = df.replace({None: np.nan, 'None': np.nan})

    # 3. Specify the numeric columns we wish to analyze and visualize
    #    (feel free to add/remove as needed from your dataset).
    columns_of_interest = [
        "player",
        "value",
        "undiscounted_value",
        "nash_welfare",
        "proposal_proportion_player_1",
        "proposal_proportion_player_2",
        "concession_size",
        "security_level_player_1",
        "security_level_player_2",
        "rawlsian_welfare",
        "gini_coefficient",
        "utilitarian_welfare",
        "jain_fairness_index",
        
    ]
    
    # 4. Convert relevant columns to numeric, coerce errors -> NaN
    for col in columns_of_interest:
        
        if col in df.columns and col not in ["game_num", "round_num", "player"]:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # 5. Compute summary statistics by round (mean, std, etc.)
    #    This will skip NaN by default (skipna=True).
    group_by_round = df.groupby("round_num")
    summary_stats = {}
    for col in columns_of_interest:
        if col in df.columns:
            summary_stats[col] = {
                "mean": group_by_round[col].mean(),
                "std":  group_by_round[col].std(),
                "min":  group_by_round[col].min(),
                "max":  group_by_round[col].max()
            }
    
    # 6. Print out some aggregated summaries
    print("=== AGGREGATE STATISTICS BY ROUND ===")
    print("Round |  Column                           |  Mean    |  Std     |  Min     |  Max")
    print("----------------------------------------------------------------------------------")
    unique_rounds = sorted(df["round_num"].dropna().unique())
    for rnd in unique_rounds:
        for col in columns_of_interest:
            if col in df.columns:
                col_stats = summary_stats[col]
                mean_val = col_stats["mean"].get(rnd, np.nan)
                std_val  = col_stats["std"].get(rnd, np.nan)
                min_val  = col_stats["min"].get(rnd, np.nan)
                max_val  = col_stats["max"].get(rnd, np.nan)
                print(f"{rnd:<5} | {col:<32} | {mean_val:8.2f} | {std_val:8.2f} | {min_val:8.2f} | {max_val:8.2f}")
    
    # 7. Create line plots for each column vs. round_num
    #    Only plot if the column actually exists in the DataFrame.
    columns_exist = [col for col in columns_of_interest if col in df.columns]
    nrows = len(columns_exist)
    fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(8, 4 * nrows), sharex=True)
    
    # If there's only one column, axes won't be an array
    if nrows == 1:
        axes = [axes]
    
    for ax, col in zip(axes, columns_exist):
        aggregated = group_by_round[col].mean()  # skipna=True by default
        ax.plot(aggregated.index, aggregated.values, marker='o')
        ax.set_title(f"Mean {col} by Round")
        ax.set_xlabel("Round Number")
        
        ax.set_ylabel(col)
    
    plt.tight_layout()
    plt.show()


analyze_bargaining_statistics(f"../experiments/pathology_results_{date}_{games}_{prompt_style}_circle_{circle}.csv")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame containing the game results
# If not already loaded, read the CSV file
df = pd.read_csv('../experiments/pathology_results_1_23_2025_100_openai_o1_maximize_value_outside_offer_cot_1_test2_circle_3.csv')

# Convert string representations of lists to actual lists if necessary
import ast
df['offer'] = df['offer'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else x)

# Create a 'turn_order' column where:
# - Player 1's moves are integers (1, 2, 3, ...)
# - Player 2's moves are at .5 increments (1.5, 2.5, 3.5, ...)
df['turn_order'] = df['round_num'] + (df['player'] - 1) * 0.5

# Sort the DataFrame by 'turn_order' to ensure correct plotting order
df.sort_values('turn_order', inplace=True)

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot a metric of interest against 'turn_order'
# Replace 'value' with the actual column you wish to plot
plt.plot(df['turn_order'], df['value'], marker='o', linestyle='-')

# Customize the x-axis
plt.xlabel('Turn Order')
plt.xticks(df['turn_order'], rotation=90)
plt.ylabel('Value')  # Replace with your metric's name
plt.title('Game Progression Over Turns')

# Optional: Add gridlines for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Show the plot
plt.tight_layout()
plt.show()

## Compute Envy Free Results By Game

In [ ]:
#Loop through envy_results_history and compute envy free results
envy_results_by_game = pd.DataFrame()
envy_results_by_game.columns = pd.DataFrame(columns=[ #Set up dataframe to store game pathology results
        "game_num", "outside_offer_consistency", "offer_increasing", "envy_free_strict", "envy_free_exclude_one_item"
    ])
for i, game in envy_results_history.items():
    print("="*60)
    print(f"Game {i + 1}:")
    evaluator = GameEvaluator(game)
    print(f"Outside offer consistency: {evaluator.evaluate_outside_offer_consistency()}")
    print(f"Offer increasing: {evaluator.evaluate_offer_increasing()}")
    print(f"Envy-free (strict): {evaluator.evaluate_envy_free()}")
    print(f"Envy-free (exclude one item): {evaluator.evaluate_envy_free(exclude_one_item=True)}")
    envy_results_by_game = pd.concat([envy_results_by_game, pd.DataFrame([{
        "game_num": i, "outside_offer_consistency": evaluator.evaluate_outside_offer_consistency(),
        "offer_increasing": evaluator.evaluate_offer_increasing(), "envy_free_strict": evaluator.evaluate_envy_free(),
        "envy_free_exclude_one_item": evaluator.evaluate_envy_free(exclude_one_item=True)
    }])], ignore_index=True)

#percentage of games that are envy free
print(f"Envy free (strict): {envy_results_by_game['envy_free_strict'].value_counts() / len(envy_results_by_game)}")
print(f"Envy free (exclude one item): {envy_results_by_game['envy_free_exclude_one_item'].value_counts() / len(envy_results_by_game)}")
print(f"Outside offer consistency: {envy_results_by_game['outside_offer_consistency'].value_counts() / len(envy_results_by_game)}")
print(f"Offer increasing: {envy_results_by_game['offer_increasing'].value_counts() / len(envy_results_by_game)}")



## Envy Free Results by Player

In [ ]:
envy_results_history_copy = envy_results_history.copy()
for i, game in envy_results_history.items():
    print("="*60)
    print(f"Game {i + 1}:")
    print(f"Game History Values:")
    for player in [0, 1]:
        player_offers = game.get_offers(player)
        player_values = game.agent_1_values if player == 0 else game.agent_2_values
        outside_value = game.agent_1_outside_value if player == 0 else game.agent_2_outside_value
        
        print(f"\nPlayer {player + 1}:")
        print(f"Outside offer value: {outside_value}")
        
        for i, offer in enumerate(player_offers):
            given_value = torch.dot(player_values, offer.offer)
            total_value = torch.dot(player_values, game.items)
            kept_value = total_value - given_value
            
            print(f"Offer {i + 1}:")
            print(f"  Offer: {offer.offer}")
            print(f"  Player values: {player_values}")
            print(f"  Value calculation:")
            print(f"    Total value: {total_value.item()} = {player_values} · {game.items}")
            print(f"    Given value: {given_value.item()} = {player_values} · {offer.offer}")
            print(f"    Kept value: {kept_value.item()} = {total_value.item()} - {given_value.item()}")
            print(f"  Value of kept items: {kept_value.item()}")

    print("\nEnvy-free evaluation details:")
    for player in [0, 1]:
        opponent = 1 - player
        player_offers = game.get_offers(player)
        opponent_offers = game.get_offers(opponent)
        player_values = game.agent_1_values if player == 0 else game.agent_2_values
        
        print(f"\nPlayer {player + 1}:")
        for i, offer in enumerate(player_offers):
            if i < len(opponent_offers):
                opp_offer = opponent_offers[i]
                player_bundle_value = torch.dot(player_values, game.items - offer.offer)
                opponent_bundle_value = torch.dot(player_values, game.items - opp_offer.offer)
                
                print(f"Round {i + 1}:")
                print(f"  Player's offer: {offer.offer}")
                print(f"  Opponent's offer: {opp_offer.offer}")
                print(f"  Player's bundle value: {player_bundle_value.item()} = {player_values} · ({game.items} - {offer.offer})")
                print(f"  Opponent's bundle value (to player): {opponent_bundle_value.item()} = {player_values} · ({game.items} - {opp_offer.offer})")
                
                if player_bundle_value >= opponent_bundle_value:
                    print("  Envy-free: Yes")
                else:
                    print("  Envy-free: No")
                    print("  Checking if removing one item makes it envy-free:")
                    for j in range(len(game_history.items)):
                        temp_opp_offer = opp_offer.offer.clone()
                        if temp_opp_offer[j] > 0:
                            temp_opp_offer[j] -= 1
                            temp_opponent_bundle_value = torch.dot(player_values, game.items - temp_opp_offer)
                            print(f"    Removing item {j + 1}: {temp_opponent_bundle_value.item()} = {player_values} · ({game.items} - {temp_opp_offer})")
                            if player_bundle_value >= temp_opponent_bundle_value:
                                print(f"    Envy-free after removing item {j + 1}: Yes")
                                break
                    else:
                        print("    Envy-free after removing any single item: No")


In [ ]:
print("Checking envy_results_history structure:")
print(f"Type: {type(envy_results_history_copy.items())}")
print(f"Length: {len(envy_results_history_copy.items())}")
if len(envy_results_history_copy.items()) > 0:
    first_game = next(iter(envy_results_history_copy.items()))[1]
    print("\nFirst game attributes:")
    print(f"Has items: {hasattr(first_game, 'items')}")
    print(f"Has agent_1_values: {hasattr(first_game, 'agent_1_values')}")
    print(f"Has agent_1_outside_value: {hasattr(first_game, 'agent_1_outside_value')}")
    print(f"Has get_offers method: {hasattr(first_game, 'get_offers')}")
    if hasattr(first_game, 'get_offers'):
        print(f"Number of offers player 1: {len(first_game.get_offers(0))}")
        print(f"Number of offers player 2: {len(first_game.get_offers(1))}")

In [ ]:
#TODO: Move this to metrics/envy_free_analysis.py
def analyze_envy_freeness_comprehensive(game_history):
    total_games = len(game_history)
    moves_data = []
    game_properties = []
    
    for game_id, game in game_history.items():
        evaluator = GameEvaluator(game)
        max_round = max(len(game.get_offers(0)), len(game.get_offers(1)))
        
        # Track game-level properties
        is_outside_consistent = evaluator.evaluate_outside_offer_consistency()
        is_offer_increasing = evaluator.evaluate_offer_increasing()
        is_final_allocation = evaluator.evaluate_envy_free()
        is_final_one_item_ef = evaluator.evaluate_envy_free(exclude_one_item=True)
        
        game_properties.append({
            'game_num': game_id,
            'rounds_played': max_round,
            'outside_consistent': is_outside_consistent,
            'offer_increasing': is_offer_increasing
        })
        
        for round_num in range(max_round):
            steps_in_round = 0
            for player in [0, 1]:
                opponent = 1 - player
                player_offers = game.get_offers(player)
                opponent_offers = game.get_offers(opponent)
                
                is_last_step = (round_num == max_round - 1 and 
                              round_num >= len(opponent_offers) - 1)
                
                if round_num < len(player_offers):
                    steps_in_round += 1
                    player_values = game.agent_1_values if player == 0 else game.agent_2_values
                    offer = player_offers[round_num]
                    
                    if round_num < len(opponent_offers):
                        opp_offer = opponent_offers[round_num]
                        player_bundle_value = torch.dot(player_values, game.items - offer.offer)
                        opponent_bundle_value = torch.dot(player_values, game.items - opp_offer.offer)
                        
                        is_strict_ef = bool(player_bundle_value >= opponent_bundle_value)
                        is_one_item_ef = False
                        
                        if not is_strict_ef:
                            for j in range(len(game.items)):
                                temp_opp_offer = opp_offer.offer.clone()
                                if temp_opp_offer[j] > 0:
                                    temp_opp_offer[j] -= 1
                                    temp_value = torch.dot(player_values, game.items - temp_opp_offer)
                                    if player_bundle_value >= temp_value:
                                        is_one_item_ef = True
                                        break
                        
                        moves_data.append({
                            'game_num': game_id,
                            'round_num': round_num + 1,
                            'step_num': steps_in_round,
                            'player': player + 1,
                            'is_strict_ef': is_strict_ef,
                            'is_one_item_ef': is_one_item_ef,
                            'is_final_allocation': is_last_step,
                            'final_strict_ef': is_final_allocation if is_last_step else False,
                            'final_one_item_ef': is_final_one_item_ef if is_last_step else False,
                            'outside_consistent': is_outside_consistent,
                            'offer_increasing': is_offer_increasing
                        })
    
    df = pd.DataFrame(moves_data)
    game_props_df = pd.DataFrame(game_properties)
    
    def print_stats(group, name):
        strict_ef_count = group['is_strict_ef'].sum()
        one_item_ef_count = group['is_one_item_ef'].sum()
        outside_consistent_count = group['outside_consistent'].sum()
        offer_increasing_count = group['offer_increasing'].sum()
        count = len(group)
        
        print(f"\n=== {name} ===")
        print(f"Count: {count}")
        print(f"Strict Envy-Free: {strict_ef_count}/{count} ({(strict_ef_count/count*100):.1f}%)")
        print(f"One-Item Envy-Free: {one_item_ef_count}/{count} ({(one_item_ef_count/count*100):.1f}%)")
        print(f"Outside Offer Consistent: {outside_consistent_count}/{count} ({(outside_consistent_count/count*100):.1f}%)")
        print(f"Offer Increasing: {offer_increasing_count}/{count} ({(offer_increasing_count/count*100):.1f}%)")
    
    print("\nGAME LENGTH STATISTICS")
    print(f"Average rounds per game: {game_props_df['rounds_played'].mean():.1f}")
    print(f"Max rounds in any game: {game_props_df['rounds_played'].max()}")
    print(f"Min rounds in any game: {game_props_df['rounds_played'].min()}")
    
    # Overall game properties
    print("\nOVERALL GAME PROPERTIES")
    print(f"Outside Offer Consistent: {game_props_df['outside_consistent'].sum()}/{total_games} ({(game_props_df['outside_consistent'].sum()/total_games*100):.1f}%)")
    print(f"Offer Increasing: {game_props_df['offer_increasing'].sum()}/{total_games} ({(game_props_df['offer_increasing'].sum()/total_games*100):.1f}%)")
    
    # Overall statistics
    print("\nOVERALL STATISTICS")
    print(f"Total Games: {total_games}")
    print(f"Total Moves: {len(df)}")
    
    # Final allocation statistics
    final_moves = df[df['is_final_allocation']]
    final_strict = final_moves.groupby('game_num')['final_strict_ef'].any()
    final_one_item = final_moves.groupby('game_num')['final_one_item_ef'].any()
    
    print("\nFINAL ALLOCATION STATISTICS")
    print(f"Strict Envy-Free: {final_strict.sum()}/{total_games} ({(final_strict.sum()/total_games*100):.1f}%)")
    print(f"One-Item Envy-Free: {final_one_item.sum()}/{total_games} ({(final_one_item.sum()/total_games*100):.1f}%)")
    
    # By Player
    for player in [1, 2]:
        player_moves = df[df['player'] == player]
        print_stats(player_moves, f"Player {player}")
    
    # By Round
    for round_num in sorted(df['round_num'].unique()):
        round_moves = df[df['round_num'] == round_num]
        print_stats(round_moves, f"Round {round_num}")
        
        # By Step within Round
        for step_num in sorted(round_moves['step_num'].unique()):
            step_moves = round_moves[round_moves['step_num'] == step_num]
            print_stats(step_moves, f"Round {round_num} - Step {step_num}")
        
        # By Player within Round
        for player in [1, 2]:
            player_round_moves = round_moves[round_moves['player'] == player]
            print_stats(player_round_moves, f"Round {round_num} - Player {player}")

    return df

envy_stats_df = analyze_envy_freeness_comprehensive(envy_results_history_copy)  
#save to csv
envy_stats_df.to_csv(f'envy_stats_df_{date}_{games}_{prompt_style}.csv', index=False)


## Statistics 

In [ ]:
# Statistics of Envy Free Results
import pandas as pd
import torch
def compute_statistics(envy_results_history):
    # Initialize dictionaries to store statistics
    game_stats = []
    round_stats = []
    player_stats = []
    
    print(f"Number of games in history: {len(envy_results_history)}")  # Debug print
    
    for game_id, game in envy_results_history.items():
        print(f"Processing game {game_id + 1}")  # Debug print
        
        # Game level statistics
        game_data = {
            'game_id': game_id + 1,
            'total_rounds': len(game.get_offers(0)),
            'player1_outside_value': game.agent_1_outside_value,
            'player2_outside_value': game.agent_2_outside_value,
            'total_items': len(game.items)
        }
        game_stats.append(game_data)
        
        # Process each player's offers and values
        for player in [0, 1]:
            player_offers = game.get_offers(player)
            player_values = game.agent_1_values if player == 0 else game.agent_2_values
            opponent = 1 - player
            opponent_offers = game.get_offers(opponent)
            
            print(f"  Player {player + 1} has {len(player_offers)} offers")  # Debug print
            
            # Player level statistics
            player_data = {
                'game_id': game_id + 1,
                'player_id': player + 1,
                'total_value': torch.dot(player_values, game.items).item(),
                'outside_value': game.agent_1_outside_value if player == 0 else game.agent_2_outside_value,
                'num_offers': len(player_offers)
            }
            player_stats.append(player_data)
            
            # Round level statistics
            for round_id, (offer, opp_offer) in enumerate(zip(player_offers, opponent_offers)):
                given_value = torch.dot(player_values, offer.offer).item()
                kept_value = torch.dot(player_values, game.items - offer.offer).item()
                opponent_bundle_value = torch.dot(player_values, game.items - opp_offer.offer).item()
                
                is_envy_free = kept_value >= opponent_bundle_value
                
                round_data = {
                    'game_id': game_id + 1,
                    'round_id': round_id + 1,
                    'player_id': player + 1,
                    'given_value': given_value,
                    'kept_value': kept_value,
                    'opponent_bundle_value': opponent_bundle_value,
                    'is_envy_free': is_envy_free
                }
                round_stats.append(round_data)
    
    print(f"\nStatistics summary:")  # Debug prints
    print(f"Number of game records: {len(game_stats)}")
    print(f"Number of player records: {len(player_stats)}")
    print(f"Number of round records: {len(round_stats)}")
    
    # Convert to DataFrames and save to CSV
    game_df = pd.DataFrame(game_stats)
    round_df = pd.DataFrame(round_stats)
    player_df = pd.DataFrame(player_stats)
    
    # Save to CSV files
    game_df.to_csv('game_statistics.csv', index=False)
    round_df.to_csv('round_statistics.csv', index=False)
    player_df.to_csv('player_statistics.csv', index=False)
    
    # Print first few rows of each DataFrame
    print("\nFirst few rows of game statistics:")
    print(game_df.head())
    print("\nFirst few rows of player statistics:")
    print(player_df.head())
    print("\nFirst few rows of round statistics:")
    print(round_df.head())
    
    return game_df, round_df, player_df

game_df, round_df, player_df = compute_statistics(envy_results_history)


In [ ]:
import sys

# Add the parent directory to sys.path
sys.path.append('../')

from game_runner import NegotitaionGame
from eval.game_evaluator import GameEvaluator
import agents.simple_agent as simple_agent
import agents.llm_agent as llm_agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dataclasses import dataclass, field

# Add the main project directory to sys.path
sys.path.append('/Users/gabesmithline/Desktop/caif_negotiation/')

# Import the notebook
import import_ipynb
%run '../test_game_eval.ipynb'  # Run the notebook directly
import torch
from utils.offer import Offer

from prompts.make_prompt import make_prompt
from metrics.visualizations import (
    plot_discounted_values,
    plot_offer_evolution,
    plot_negotiation_gap,
    plot_fairness
)

# Initialize the DataFrame to store pathology results
pathology_results = pd.DataFrame()  # Store total game pathology results
# Add COLUMNS
envy_results_history = {}

## Discounted Values Over Time

For each offer/counter-offer:
- Blue line shows what Player 1 would receive from that division of items in their offer
- Red line shows what Player 2 would receive from that division of items counter offer


So each point on the graph shows the value each player would get if that particular division of items was accepted, after applying the time discount ($\gamma^{round-1}$).



In [ ]:
plot_discounted_values(rounds, p1_values, p2_values, max_rounds)


## Offer Evolution Plot

This visualization shows how the distribution of items changes over the negotiation rounds, with a separate subplot for each item type.

### Layout
- Creates a grid of subplots (2 columns)
- One subplot per item type
- For odd number of items, removes unused subplot

### Per Item Subplot
- X-axis: Negotiation rounds
- Y-axis: Number of units offered to P2
- Blue line (○): P1's offers
- Red line (○): P2's counter-offers
- Y-axis range: 0 to maximum units available for that item
- Grid shows valid integer unit values

### Interpretation
- Shows how many units of each item type each player offered to give to P2
- P1's remaining units = total_items - offered_units
- Converging lines suggest agreement being reached
- Diverging lines suggest areas of contention

In [ ]:
plot_offer_evolution(game, rounds, p1_offers, p2_offers)

In [ ]:
def get_final_division(game):
    if game.current_offer: 
        p2_items = game.current_offer.offer  
        p1_items = [t - g for t, g in zip(game.items, p2_items)]  
        return {
            'P1': p1_items,
            'P2': p2_items
        }
    return None
evaluator = GameEvaluator(game)
outside_offer_rational = evaluator.evaluate_outside_offer_consistency()

if game.current_offer:
    final_division = get_final_division(game)
    value_gaps = [abs(p1 - p2) for p1, p2 in zip(p1_values, p2_values)] 
    fairness = [(p1/(p1+p2), p2/(p1+p2)) for p1, p2 in zip(p1_values, p2_values)]  

    print("\nBasic Metrics:")
    print(f"Final P1 Value: {p1_values[-1]:.2f}")
    print(f"Final P2 Value: {p2_values[-1]:.2f}")
    print(f"Final Value Gap: {value_gaps[-1]:.2f}")
    print(f"Final Fairness Ratio: {fairness[-1][0]:.2f} : {fairness[-1][1]:.2f}")
    print("\nFinal Division of Items:")
    if final_division:
        print(f"Total Items Available: {game.items}")
        print(f"P1 Received: {final_division['P1']}")
        print(f"P2 Received: {final_division['P2']}")
    else:
        print("No deal reached - no items were divided")

print("\nNegotiation Evaluation:")
print(f"Outside Offer Rational: {'Yes' if outside_offer_rational else 'No'}")

print(f"P1 Valuation of Each Item: {game.player_values[0]}")
print(f"P2 Valuation of Each Item: {game.player_values[1]}")

## Check Pathologies
- Check if Pathologies Occured
- Check Count in game 
- Save to csv
